In [1]:
from clickhouse_driver import Client

client = Client(host='localhost')

In [2]:
client.execute('SHOW DATABASES')

[('_temporary_and_external_tables',),
 ('default',),
 ('replica',),
 ('shard',),
 ('system',)]

In [3]:
### Загружаем тестовые данные (1 строчку)

In [4]:
%%timeit -n5 -r5
client.execute('INSERT INTO default.test (id, user_id, movie_id, viewed_frame) VALUES', [(1, 'id_1', 'film_id_1', 120)])

The slowest run took 7.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1.68 ms ± 1.36 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [5]:
%%timeit -n5 -r5
client.execute('SELECT * FROM default.test WHERE id=1')

The slowest run took 4.42 times longer than the fastest. This could mean that an intermediate result is being cached.
4.66 ms ± 2.91 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


### Параллельно запускаем загрузку данных

### Загружаем тестовые данные (10 млн записей)

In [6]:
import csv

values = []
with open('../test_data/first_part.csv', "r", newline='') as file:
    file_data = csv.DictReader(file)
    for idx, line in enumerate(file_data, start=1):
        line['id'] = idx
        line['viewed_frame'] = int(line['viewed_frame'])
        values.append(line)

In [7]:
%%time
client.execute('INSERT INTO default.test (id, user_id, movie_id, viewed_frame) VALUES', values)

CPU times: user 16.8 s, sys: 310 ms, total: 17.1 s
Wall time: 17.1 s


10000000

In [8]:
%%timeit -n5 -r5
client.execute('SELECT count() FROM default.test')

1.96 ms ± 561 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [9]:
%%timeit -n5 -r5
client.execute('SELECT uniqExact(movie_id) FROM default.test')

1.12 s ± 13.2 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [10]:
%%timeit -n5 -r5
client.execute('SELECT uniqExact(user_id) FROM default.test')

1.05 s ± 32.7 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [11]:
%%timeit -n5 -r5
client.execute(
    """
    SELECT 
        user_id, 
        uniqExact(movie_id) 
    FROM default.test
    GROUP by user_id
    """
)

1.72 s ± 21.8 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [12]:
%%timeit -n5 -r5
client.execute(
    """
    SELECT
        user_id,
        sum(viewed_frame),
        max(viewed_frame)
    FROM default.test
    WHERE user_id='id_5'
    GROUP by user_id
    """
)

651 ms ± 5.07 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [13]:
%%timeit -n5 -r5
client.execute(
    """
    SELECT 
        user_id, 
        sum(viewed_frame),
        max(viewed_frame) 
    FROM default.test
    GROUP by user_id
    """
)

754 ms ± 2.62 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
